In [5]:

import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karthik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [135]:

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv(r"C:\Users\karthik\Downloads\tweets_1.csv")


In [7]:
x= df['text'][df['Unnamed: 0']==2]
print(x)

2    The media and establishment want me out of the race so badly -  I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA
Name: text, dtype: object


In [136]:
df.head(10)

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu,DonaldTrump
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA",DonaldTrump
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz,DonaldTrump
5,5,4h4 hours ago,786950598826532864,/realDonaldTrump/status/786950598826532864,False,WHAT THEY ARE SAYING ABOUT THE CLINTON CAMPAIGN’S ANTI-CATHOLIC BIGOTRY: \nhttp://bit.ly/2dcbtvkCrooked,DonaldTrump
6,6,Oct 13,786589172911964160,/realDonaldTrump/status/786589172911964161,False,Thank you! #MAGA #AmericaFirstpic.twitter.com/fG313wjlKm,DonaldTrump
7,7,Oct 12,786340623804751872,/realDonaldTrump/status/786340623804751872,False,"I will be in Cincinnati, Ohio tomorrow night at 7:30pm- join me! #OhioVotesEarly #VoteTrumpPence16 \nTickets: https://www.donaldjtrump.com/schedule/register/cincinnati-oh2/ …pic.twitter.com/XUFuGc4Fg5",DonaldTrump
8,8,Oct 12,786201435486781440,/realDonaldTrump/status/786201435486781440,False,Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system!,DonaldTrump
9,9,Oct 11,786054986534969344,/realDonaldTrump/status/786054986534969344,False,Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out & #VoteTrumpPence16 on 11/8! #MAGApic.twitter.com/e90a2AL7vO,DonaldTrump


In [9]:

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [145]:
df['text'] = df['text'].astype(str)

In [146]:
df['text'] = df['text'].astype(str)
df['cleaned'] = df['text'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)


In [13]:
EMBEDDING_FILE = r'C:\Users\karthik\Downloads/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)


In [14]:
#ssplit en plusieurs mots la description cleaned

import time
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [15]:
# Entrainement surr google model
%time
google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)



google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

Wall time: 0 ns


(0, 0)

In [16]:
google_model.wv.most_similar(positive=["steve"]) #model google des mots reconus on peut tester des trucs ex: ok, hello, etc

[('dave', 0.7480320334434509),
 ('jeff', 0.6910731196403503),
 ('jason', 0.6878510117530823),
 ('robert', 0.6830392479896545),
 ('george', 0.6792417764663696),
 ('kevin', 0.676276683807373),
 ('todd', 0.6758198738098145),
 ('ryan', 0.6735630035400391),
 ('greg', 0.670603334903717),
 ('michael', 0.6679339408874512)]

In [22]:
# genere average word2dec pour chaque champ
def vectors(x):
    
    
    global word_embeddings
    word_embeddings = []

    
    for line in df['cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec + google_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)
    
    return len(word_embeddings)


In [23]:
vectors(df)

C:\Users\karthik\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
C:\Users\karthik\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


17216

In [24]:
def vectors1(text):
    
    
    global word_embeddings1
    word_embeddings1 = []

    
    for line in text:
        avgword2vec1 = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec1 is None:
                    avgword2vec1 = google_model[word]
                else:
                    avgword2vec1 = avgword2vec1 + google_model[word]
                
        if avgword2vec1 is not None:
            avgword2vec1 = avgword2vec1 / count
        
            word_embeddings1.append(avgword2vec1)
    return len(word_embeddings1)

In [27]:
vectors1("Certainly has been an interesting 24 hours!")


C:\Users\karthik\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


35

In [33]:
text = "Certainly has been an interesting 24 hours!"
a= _removeNonAscii(text)
print(a)
print("ok")
a= make_lower_case(a)
print(a)


a= remove_stop_words(a)
   
print(a)

a= remove_html(a)
print(a)

a = remove_punctuation(a)
print(a)

Certainly has been an interesting 24 hours!
ok
certainly has been an interesting 24 hours!
certainly interesting 24 hours!
certainly interesting 24 hours!
certainly interesting 24 hours


In [ ]:
vectors(df)
vectors1("interested way of looking")
cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)
cosine_similarities1 = np.mean(cosine_similarity(word_embeddings1, word_embeddings1))
print(cosine_similarities.shape)
print("\n ok")
print(cosine_similarities1)
print("\n okkkk")

print("\nmean of arr, axis = 1 : ", cosine_similarities1)

# sim_scores = list(enumerate(cosine_similarities1[5],1))
# print(sim_scores)

# indices = pd.Series(df.index, index = df['text']).drop_duplicates()
# #     print(indices)     simple dataframe comme df
# idx = indices["New York Fashion Week is really bad, and used to be so glamorous and exciting! No stars, no fun-just boring. They need serious help. #NYFW"]   #l'id ou la position dans le dataframe du tweet
    
# sim_scores = list(enumerate(cosine_similarities[idx]))
# print(sim_scores)

In [147]:
df.tail()

,Unnamed: 0,date,id,link,retweet,text,author,cleaned
17212,17212,8 May 2009,1741160716,/realDonaldTrump/status/1741160716,False,New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e,DonaldTrump,new blog post celebrity apprentice finale lessons learned along way http tinyurl com qlux5e
17213,17213,8 May 2009,1737479987,/realDonaldTrump/status/1737479987,False,Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!,DonaldTrump,donald trump reads top ten financial tips late show david letterman http tinyurl com ooafwn funny
17214,17214,4 May 2009,1701461182,/realDonaldTrump/status/1701461182,False,Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!,DonaldTrump,donald trump appearing view tomorrow morning discuss celebrity apprentice new book think like champion
17215,17215,4 May 2009,1698308935,/realDonaldTrump/status/1698308935,False,Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!,DonaldTrump,sure tune watch donald trump late night david letterman presents top ten list tonight
17216,17216,dateeclate,1,/realDonaldTrump/status/1784884848,False,fucking idiot 23 Hours!,DonalTrump,fucking idiot 23 hours


In [247]:

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv(r"C:\Users\karthik\Downloads\tweets_1.csv")


In [248]:
def essentiel(df,text):
    df = df.append({'Unnamed: 0': 17216,'date':"dateeclate",'id':1,'link':"/realDonaldTrump/status/1784884848", 'retweet':"False", 'text':text,'author':"DonalTrump"}, ignore_index=True)
    df['text'] = df['text'].astype(str)
    df['cleaned'] = df['text'].apply(_removeNonAscii)
    df['cleaned'] = df.cleaned.apply(func = make_lower_case)
    df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
    df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
    df['cleaned'] = df.cleaned.apply(func=remove_html)
    return (df)
    

In [249]:
df = essentiel(df,"Love you")

In [251]:


def recommendations(text):
    
#     df = df.loc[17216]= [17216, "date", 1, "/realDonaldTrump/status/1784884848","False",text ,"DonaldTrump"]
#     df.append({'Unnamed: 0': 17216,'date':"dateeclate",'id':1,'link':"/realDonaldTrump/status/1784884848", 'retweet':"False", 'text':text,'author':"DonalTrump"}, ignore_index=True)
#     data.tail()
    

    vectors(df)
    
#     vectors1(text)

    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)
#     cosine_similarities1 = cosine_similarity(word_embeddings1, word_embeddings1)

  
    books = df[['text','id']]
 
    indices = pd.Series(df.index, index = df['text'])
#     print(indices)     #simple dataframe comme df
    idx = indices[text]   #l'id ou la position dans le dataframe du tweet
    print(idx)
    sim_scores = list(enumerate(cosine_similarities[idx]))
    print(sim_scores)
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

    sim_scores = sim_scores[1:20]

    book_indices = [i[0] for i in sim_scores]
    recommend = books.iloc[book_indices]

    print(sim_scores)
    return(recommend)

In [253]:
recommendations("Love you")

C:\Users\karthik\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
C:\Users\karthik\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


17216
[(0, 0.18489562), (1, 0.33456326), (2, 0.34114984), (3, 0.31105754), (4, 0.42730054), (5, 0.19834566), (6, 0.3901446), (7, 0.23322552), (8, 0.21468633), (9, 0.441494), (10, 0.29407865), (11, 0.4325285), (12, 0.326311), (13, 0.3693089), (14, 0.19566703), (15, 0.30200884), (16, 0.36973363), (17, 0.21015841), (18, 0.19954723), (19, 0.2661419), (20, 0.31221512), (21, 0.41484296), (22, 0.25652984), (23, 0.2702531), (24, 0.23011044), (25, 0.24575788), (26, 0.32125813), (27, 0.32429576), (28, 0.41165584), (29, 0.22790101), (30, 0.2405269), (31, 0.35353115), (32, 0.26043525), (33, 0.256832), (34, 0.29627436), (35, 0.2250711), (36, 0.40756524), (37, 0.2605275), (38, 0.28450248), (39, 0.30371058), (40, 0.3905341), (41, 0.4267624), (42, 0.36533165), (43, 0.38324502), (44, 0.37477514), (45, 0.29585442), (46, 0.24047408), (47, 0.28873596), (48, 0.28858787), (49, 0.41749218), (50, 0.2346941), (51, 0.26999444), (52, 0.39362928), (53, 0.3964417), (54, 0.23103687), (55, 0.28394774), (56, 0.446007

,text,id
9983,@BrikMillerEDG You will love it!,357509419464003584
12541,@RealKyleMorris You will love it!,287230122526924800
11729,"""@Candynecklace2: @realDonaldTrump I love love love Mr. Trump he is my hero "" Thanks.",307337362634076160
8862,"""@_stacys_mom_: I just love @realDonaldTrump."" Very nice.",394356629384482816
12548,@michelle88fan I love what @GovernorCorbett is doing.,286949964746211328
5392,"""@biggangVH1: @realDonaldTrump love the show! #CelebApprentice""",551935679459250176
5363,"""@loribuckmajor: @realDonaldTrump @BrandiGlanville is as real as they come. Love her #CelebApprentice""",551953704241618944
5064,"""@MelRivers: @realDonaldTrump @IvankaTrump @DonaldJTrumpJr Thank you for all of your kindness and support. My mother loved you all."" LOVE!",557405593661890560
5999,"""@LenWDougleman: Dear Donald, I love you success aftershave. I love to walk out of the house with the confidence that I smell of Trump.""",530704729731784704
13489,.@brandonhardest Love what you do and work hard.,250961000759693312
